
   
<img src="halite_graphic.png" alt="HALITE" width="200" height="200">

# Halite Modeling Framework
*NP+RS Turley*
*7/31/2020*

Kaggle's Halite competition invites competitors to submit Python code to compete in a 4-person space mining battle. For our entry, we want to create a framework that is flexible enough to describe the range of submitted strategies. By observing our opponents actions, we can then approximate their strategy to predict their next moves. We could also use this framework to define the optimal strategies of our own practice bots...or maybe even our competitive entries.

### Outline  

  **A. Modeling Framework Overview**  
  **B. Ship Tactics and Paths**  
  **C. State Variables**  
  **D. Expectations**  
  **E. Formal Strategy**  



## A. Modeling Framework Overview


Our modeling framework suggests how the team will select actions each turn given their information. It is helpful to see the actions as the result of a framework where the team's **Strategy** assigns **Tactics** to each ship, with a **Path** function determining which actions best accomplish the tactical objective. 

For each ship, there are normally many ways to accomplish the tactical objective, so we need to determine the best path for the ship to take.

**Ship Tactic &#10142; Path(Location, State, Expectations) &#10142; Next Ship Action**
* The **Path** function for a given tactic determines the best actions to take from the current location (given the state and expectations of other players) and suggests the best next action
  * the **Location** is the ship's current position on the board
  * The **Next Ship Action** will be one of the 6 actions defined in the game [N, S, E, W, Mine, Convert]

**Strategy(State,Expectations) &#10142; Tactics**  

* The **Strategy** is a mapping from the **state** variables describing the game and the **expectations** of other players' actions to assign tactics to each ship and shipyard
  * The **State** is a set of key variables defined from past and present game information that we feel are especially relevant
  * The **Expectations** are the probabilities assigned to future actions taken by other players
  * The **Tactics** define the short-term objective


Note that shipyards can only receive one action from the strategy (Spawn), so there is no need for tactics and paths.


## B. Ship Tactics

**Basic Tactics**  

We will define 6 basic ship-level tactics

|           | Tactic      | Goal                                   |
| --------- | ----------- | -------------------------------------- |
| &#10102;  | Mine Halite | Travel to and mine a specific cell on the board   |
| &#10103;  | To Shipyard | Travel to closest friendly shipyard    |
| &#10104;  | Evade       | Avoid nearby enemy ships               |
| &#10105;  | Attack Ship | Intercept and attack enemy ship        |
| &#10106;  | Attack Yard | Travel to and attack enemy shipyard    |
| &#10107;  | Convert     | Travel to cell and convert to shipyard |


**Complex Tactics**

A more complex strategy could instruct advanced tactics that involve multiple ships, but they are not currently in scope. However, we will want to consider them in designing our own strategies. Examples of these include sending clusters of ships in an attack, luring enemy ships to a "honey pot" while other ships lie in wait, blockades of enemy shipyards, etc.




**Paths**
  
Once each ship has received its assigned tactical objective, it needs to select specfic actions from the six defined in Halite: move *North*, move *South*, move *East*, move *West*, stay and *Mine*, or *Convert* to shipyard. The ships **path** considers state variables and expectations to suggest the best sequence of subsequent ship actions to accomplish the objective. 

Since movement is only allowed in the four cardinal directions (N/S/E/W) there will typically be multiple equidistant paths to the same destination. Consider the image below showing three paths with equal Manhattan distances.

![manhattan_dist](manhattan_dist.png)

The path function suggests whether the red, yellow or blue (or another!) path are the best to take to a given destination.

The path selection will consider:
* shortest route (fewest actions)
* avoid crashes with friendly ships
* avoid interception from "lean" enemy ships (carrying less halite)
* encourage interception of "fat" enemy ships (carrying more halite)
* stay close to friendly lean ships
* include randomness to make movement less predictable

The path function weighs each factor in a manner defined by the strategy. It is important to note that the path considers a sequence of steps--but the output is a set of probabilities for the next action. In a deterministic case, the output could simply be an action, but we want to leave open the ability to purposefully add randomness or allow uncertainty to propagate in the model.

The ship's actions in subsequent turns may be as the path imagined, but a change in tactic or updated expectations could lead to to a path adjustment.

*Path score formula*

Input: StartPoint, EndPoint
Parameters:
Output: probabilities for next action

## C. State Variables

The state variables should include all of the information that allow the strategy to decide the optimal tactics for each ship/shipyard. They are also important inputs of the path function. Their definition is a key contribution to the model. 

Of course, the state variables could simply be a raw list of all game info, with the full player board and all past moves. However, the work of defining the relevant variables from the the board is itself a modeling contribution. For better or worse, it channels (and potentially limits) the informtion a strategy employs. With intelligent state variables, we can more easily build strategies manually and interpret strategies built by machines. 

We propose the list below that will form the Python dictionary State


| Category              |    | State Variable                        | Python Attribute Name
| --------------------- | -- | ------------------------------------- | -----------------
| Game Status           | 1  | Turns left in game                    | `State['StepsLeft']`
|                       | 2  | Total board halite                    | `State['TotalHalite']`
| For each team         | 3  | Halite stored                         | `State['Team#']['TeamHalite']`
|                       | 4  | Halite in cargo                       | `State['Team#']['TeamCargo']`
|                       | 4  | Number Ships                          | `State['Team#']['TeamShips']`
|                       | 5  | Number of shipyards                   | `State['Team#']TeamShipyards`
|                       | 6  | Cells controlled                      | `State['Team#']TeamCellControl`
|                       | 7  | Halite in cells controlled            | `State['Team#']TeamHaliteControl`
|                       | 8  | Shipyard kills                        | `State['Team#']TeamShipyardKills`
|                       | 9  | Ship kills                            | `State['Team#']TeamShipKills`
| For each ship         | 10 | last two moves                        | `State['Team#']['Ship#']PastActions`
|                       | 11 | last tactic probabilities             | `State['Team#']['Ship#']Tactic` 
|                       | 13 | distance to shipyard of each team     | `State['Team#']['Ship#']YardDistTeam#`
|                       | 15 | distance to key destinations?         | 
|                       | 16 | 2 move grid of dangerous ships        | `State['Team#']['Ship#']['YardDistTeam#EvadeGrid']`
|                       | 17 | path of fat target ships              | `State['Team#']['Ship#']['YardDistTeam#InterceptGrid']`
| For each shipyard     | 18 | enemies at distance one               | `State['Team#']['Shipyard#']['ShipyardEnemyD1']`
|                       | 19 | enemies at distance two               | `State['Team#']['Shipyard#']['ShipyardEnemyD2']`
|                       | 19 | friends at distance zero              | `State['Team#']['Shipyard#']['ShipyardFriendD0']`
|                       | 20 | friends at distance one               | `State['Team#']['Shipyard#']['ShipyardFriendD1']`
|                       | 21 | friends at distance two               | `State['Team#']['Shipyard#']['ShipyardFriendD2']`
| For the Board         | 22 | halite on cell                        | `State['Board']['HaliteD0']`
|                       | 23 | halite in one move radius             | `State['Board']['HaliteD1']`
|                       | 24 | halite in two move radius             | `State['Board']['HaliteD2']`
|                       | 25 | ship from team[#] at distance=0       | `State['Board']['ShipD0']['Team#']`
|                       | 26 | ship from team[#] at distance=1       | `State['Board']['ShipD1']['Team#']`
|                       | 27 | ship from team[#] at distance=2       | `State['Board']['ShipD2']['Team#']`

The concept of "key locations" reduces the number of cells of interest that the strategy and tactics will consider. They are defined to include cells with a minimal halite value (e.g. halite >4) that are the current location of a ship or meet any of the six following criteria:
1. eight highest halite cells on the board
2. four highest halite cells on board with no ships in 3-move radius
3. highest halite cell in one move for each ship
4. the highest halite cell within three moves of each ship
5. the 2nd highest halite cell within 3 moves of each ship (not in the same direction as 1st)
6. the 3rd highest halite cell within 3 moves of each ship (not in the same directions as 1st or 2nd)

If a cell meets one of these six criteria, it will not be removed from the list of key locations unless it does not meet the criteria for two subsequent turns.

## D. Expectations


There are four potential types of expectations we can consider:
* momentum - assume that enemy ships are going to continue in the direction they moved last turn
* default(s) - use the actions that would be submitted by the most popular/successful publicly available bot (currently Optimus Mining)
* random - assume that the enemy is equally likely to make any action
* strategic learning - use this same framework and learn the enemies parameters from their observed actions

*Strategic learning requires a recursive relationship (i.e. team A's strategy depends what they believe Team B believes about what what they believe...). We may omit this or at least limit it to just one level of recursion.*

**Random**

Random movement is a ridiculous expectation, but since we are modeling the expecttions of other teams, we need to include the possibility that they have simplistic views of enemy ship movement. Also, if the full expectatoin is a weighted mix of these approaches, adding weight to the expectation of random movement proxies for general uncertainty.

**Momentum**

The momentum expectation simply expects that the ship will continue in the direction implied by the last two turns, or if mining, reversing direction.

| Last Two Moves     | Predictions       |                                  |
| ------------------ | ----------------- | -------------------------------- |
| &#8594; , &#8594;  | &#8594; , &#8594; | Continue path                    |
| &#8593; , &#8594;  | &#8594; , &#8593; | Continue path (toward diagonal)  |
| &#8595; , &#8594;  | &#8594; , &#8595; | Continue path (toward diagonal)  |
| &#8592; , &#8594;  | &#8594; , &#8594; | Continue reverse direction       |
| &#10226;, &#8594;  | &#8594; , &#8594; | Continue path after mining       |
| &#8594;, &#10226;  | &#8689; , &#8689; | Go toward their base             |
| &#10226;, &#10226; | &#8689; , &#8689; | Go toward their base             |

And with uncertainty, we can consider diagonal predictions as having 75/25 and then 50/50 uncertainty for the first and second move and for the "go toward their base" expectation, the movement can be weighted across all directions with the most going in the two directions that are towards the closest base.


**Default**

A large number of bots in the competition are based on the most successful public notebook(s). Since these are public, we could simply run their action decisions as the expectation, and the weight on the default is essentially a view that we know who we are up against!

**Strategic learning**

It may not be worth the computational cost, but the highest levels of play we might want to form expectations of opponents based on their own strategic model.



## E. Formal Strategy

*The following is version 1.0 and will certainly change over time!

### Strategic Parameters/Functions

* Strategy Parameters
  * Shipyard spawning
  * 
  
* Strategy Functions
  * Ship value function

**Strategy Parameters**

| Category              |    | Parameter                             | Python Name                             | Default  | Suggested
| --------------------- | -- | ------------------------------------- | --------------------------------------- | -------- | --------
| Shipyard spawning     | 1  | Target number of ships                | `PARAM.ShipyardSpawnTarget`             |  26      | 100
|                       | 2  | Adjustment if step>200                | `PARAM.ShipyardSpawnTarget200adj`       |  -2      |  0
|                       | 2  | Adjustment if step>300                | `PARAM.ShipyardSpawnTarget300adj`       |  -2      |  0
|                       | 2  | Adjustment if step>350                | `PARAM.ShipyardSpawnTarget350adj`       |  -2      |  0
|                       | 2  | Adjustment if step>380                | `PARAM.ShipyardSpawnTarget380adj`       |  -16     |  0
|                       | 2  | Scale calculated ship value         | `PARAM.ShipyardSpawnHarvestRate`          |  0.0     |  1.0





**Ship Value Function**
A key feature in any strategy concerns the value of a ship. In order to make decisions regarding spawning, attack, sacrifice, etc., we need some concept of the value of a ship as measured in halite. Roughly speaking, the value of the ship is equal to the amount of halite it can mine over its expected life. 

We could derive a formula or calibrate this to actual game data as a function of the following factors:

Factors that affect ship value
* steps left in game
* total halite on the board
* halite in reachable distance on board
* location of the ship
* ability to defend loss of shipyard
* expected life before being destroyed by enemy
* ability to destroy enemy shipyard (+500)
* ability to protect home base (+500)

Plan: go to high-level games (score about 1100?) and build dataset on the halite mined/stolen and delivered by every ship in the game as a function of predictor variables--> you've got a model.

**Shipyard**

Assume agent will create ships if has enough halite and current ships is less than target_ships
Target ships is a descending function of the steps, an increasing function of the total halite, and a decreasing function of the number of steps left
The agent will also create a new ship to defend a shipyard from being destroyed
The agent will not spawn a new ship if a an existing ship is going to shipyard
(note that an existing ship should not go to shipyard if priority is ship construction!)

The basic target for number of ships simply sets a value that decreases as the steps approach the end of the game

Target_Ships = `PARAM.ShipTarget` + `PARAM.ShipTarget200adj` $\times$ (if `State['StepsLeft']` < 200) 
                 + `PARAM.ShipTarget300adj` $\times$ (if `State['StepsLeft']` < 100) + `PARAM.ShipTarget350adj` $\times$ (if `State['StepsLeft']` < 50)
                 + `PARAM.ShipTarget390adj` $\times$ (if `State['StepsLeft']` < 10)

(*Can we do a simpler, continuous version without so many parameters that is approximately the same?*)

The second limit is to produce a ship if its expected value is greater than 500. 


Defensive and strategic considerations: 
If probability of enemy base attack is greater than 25% and enemy has >0 halite, spawn a ship
If adjacent enemy ship has no halite and probabilty of base attack next turn is greater than 50%, spawn new ship

Don't spawn if in last few turns!

Shipyard Actions
Spawn a ship in the shipyard if

In [ ]:
def shipyard_actions(board):
  #spawn a ship as long as there is no ship already moved to this shipyard
  for sy in me.shipyards:
    if turn.num_ships < turn.max_ships:
      if (turn.total_halite >= 500) and (sy.position not in turn.taken):
        #spawn one
        sy.next_action = ShipyardAction.SPAWN
        turn.taken[sy.position]=1
        turn.num_ships+=1
        turn.total_halite-=500

In [ ]:
from kaggle_environments import evaluate, make
from kaggle_environments.envs.halite.helpers import *

env = make("halite", debug=True)

In [ ]:




def dist(p1,p2):
    #Manhattan distance of the Point difference a to b, considering wrap around
    deltaX, deltaY=p2 - p1
    if abs(deltaX)>size/2:
        #we wrap around
        if deltaX<0:
            deltaX+=size
        elif deltaX>0:
            deltaX-=size
    if abs(deltaY)>size/2:
        #we wrap around
        if deltaY<0:
            deltaY+=size
        elif deltaY>0:
            deltaY-=size
    return deltaX, deltaY






In [ ]:
%%writefile submission.py
import time
import copy
import sys
import math
import collections
import pprint
import numpy as np
import scipy.optimize
import scipy.ndimage
from kaggle_environments.envs.halite.helpers import *
import kaggle_environments
import random
'''


'''
CONFIG_MAX_SHIPS=28

#print('kaggle version',kaggle_environments.__version__)
#### Global variables
all_actions=[ShipAction.NORTH, ShipAction.EAST,ShipAction.SOUTH,ShipAction.WEST]
all_dirs=[Point(0,1), Point(1,0), Point(0,-1), Point(-1,0)]
start=None
num_shipyard_targets=4
size=None
# Will keep track of whether a ship is collecting halite or carrying cargo to a shipyard
ship_target={}   # where to go to collect
me=None
did_init=False
quiet=False
C=None
class Obj:
  pass
# will hold global data for this turn, updating as we set actions.
# E.g. number of ships, amount of halite
# taking into account the actions set so far.  Also max_ships, etc.
turn=Obj()

prev_turn=Obj()


### Data
#turns_optimal[CHratio, round_trip_travel] for mining
#See notebook on optimal mining kaggle.com/solverworld/optimal-mining
turns_optimal=np.array(
  [[0, 2, 3, 4, 4, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8],
   [0, 1, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7],
   [0, 0, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7],
   [0, 0, 1, 2, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6],
   [0, 0, 0, 1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6],
   [0, 0, 0, 0, 0, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5],
   [0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

#### Functions
def print_enemy_ships(board):
  print('\nEnemy Ships')
  for ship in board.ships.values():
    if ship.player_id != me.id:
      print('{:6}  {} halite {}'.format(ship.id,ship.position,ship.halite))
      
def print_actions(board):
  print('\nShip Actions')
  for ship in me.ships:
    print('{:6}  {}  {} halite {}'.format(ship.id,ship.position,ship.next_action,ship.halite))
  print('Shipyard Actions')
  for sy in me.shipyards:
    print('{:6}  {}  {}'.format(sy.id,sy.position,sy.next_action))

def print_none(*args):
  pass

def compute_max_ships(step):
  #This needs to be tuned, perhaps based on amount of halite left
  if step < 200:
    return CONFIG_MAX_SHIPS
  elif step < 300:
    return CONFIG_MAX_SHIPS-6
  elif step < 350:
    return CONFIG_MAX_SHIPS-8
  else:
    return CONFIG_MAX_SHIPS-10

def set_turn_data(board):
  #initialize the global turn data for this turn
  turn.num_ships=len(me.ships)
  turn.max_ships=compute_max_ships(board.step)
  turn.total_halite=me.halite
  #this is matrix of halite in cells
  turn.halite_matrix=np.reshape(board.observation['halite'], (board.configuration.size,board.configuration.size))
  turn.num_shipyards=len(me.shipyards)
  #compute enemy presence and enemy halite matrices
  turn.EP,turn.EH,turn.ES=gen_enemy_halite_matrix(board)
  #filled in by shipid as a ship takes up a square
  turn.taken={}
  turn.last_episode = (board.step == (board.configuration.episode_steps-2))
    
def set_prev_turn_data(prev_board):
  #initialize the global turn data for this turn
  prev_turn.num_ships=len(prev_board.current_player.ships)
  prev_turn.max_ships=compute_max_ships(prev_board.step)
  prev_turn.total_halite=prev_board.current_player.halite
  #this is matrix of halite in cells
  prev_turn.halite_matrix=np.reshape(prev_board.observation['halite'], (prev_board.configuration.size,prev_board.configuration.size))
  prev_turn.num_shipyards=len(prev_board.current_player.shipyards)
  #compute enemy presence and enemy halite matrices
  prev_turn.EP,prev_turn.EH,prev_turn.ES=gen_enemy_halite_matrix(prev_board)
  #filled in by shipid as a ship takes up a square
  prev_turn.taken={}
  prev_turn.last_episode = (prev_board.step == (prev_board.configuration.episode_steps-2))
  
def init(obs,config):
  #This is only called on first call to agent()
  #Do initalization things
  global size
  global print
  if hasattr(config,'myval') and config.myval==9 and not quiet:
    #we are called locally, so leave prints OK
    pass
  else:
    #we are called in competition, quiet output
    print=print_none
    pprint.pprint=print_none
  size = config.size

def limit(x,a,b):
  if x<a:
    return a
  if x>b:
    return b
  return x
  
def num_turns_to_mine(C,H,rt_travel):
  #How many turns should we plan on mining?
  #C=carried halite, H=halite in square, rt_travel=steps to square and back to shipyard
  if C==0:
    ch=0
  elif H==0:
    ch=turns_optimal.shape[0]
  else:
    ch=int(math.log(C/H)*2.5+5.5)
    ch=limit(ch,0,turns_optimal.shape[0]-1)
  rt_travel=int(limit(rt_travel,0,turns_optimal.shape[1]-1))
  return turns_optimal[ch,rt_travel]

def halite_per_turn(carrying, halite,travel,min_mine=1):
  #compute return from going to a cell containing halite, using optimal number of mining steps
  #returns halite mined per turn, optimal number of mining steps
  #Turns could be zero, meaning it should just return to a shipyard (subject to min_mine)
  turns=num_turns_to_mine(carrying,halite,travel)
  if turns<min_mine:
    turns=min_mine
  mined=carrying+(1-.75**turns)*halite
  return mined/(travel+turns), turns
  
def move(pos, action):
  ret=None
  #return new Position from pos when action is applied
  if action==ShipAction.NORTH:
    ret=pos+Point(0,1)
  if action==ShipAction.SOUTH:
    ret=pos+Point(0,-1)
  if action==ShipAction.EAST:
    ret=pos+Point(1,0)
  if action==ShipAction.WEST:
    ret=pos+Point(-1,0)
  if ret is None:
    ret=pos
  #print('move pos {} {} => {}'.format(pos,action,ret))
  return ret % size

def dirs_to(p1, p2, size=21):
  #Get the actions you should take to go from Point p1 to Point p2
  #using shortest direction by wraparound
  #Args: p1: from Point
  #      p2: to Point
  #      size:  size of board
  #returns: list of directions, tuple (deltaX,deltaY)
  #The list is of length 1 or 2 giving possible directions to go, e.g.
  #to go North-East, it would return [ShipAction.NORTH, ShipAction.EAST], because
  #you could use either of those first to go North-East.
  #[None] is returned if p1==p2 and there is no need to move at all
  deltaX, deltaY=p2 - p1
  if abs(deltaX)>size/2:
    #we wrap around
    if deltaX<0:
      deltaX+=size
    elif deltaX>0:
      deltaX-=size
  if abs(deltaY)>size/2:
    #we wrap around
    if deltaY<0:
      deltaY+=size
    elif deltaY>0:
      deltaY-=size
  #the delta is (deltaX,deltaY)
  ret=[]
  if deltaX>0:
    ret.append(ShipAction.EAST)
  if deltaX<0:
    ret.append(ShipAction.WEST)
  if deltaY>0:
    ret.append(ShipAction.NORTH)
  if deltaY<0:
    ret.append(ShipAction.SOUTH)
  if len(ret)==0:
    ret=[None]  # do not need to move at all
    
  if len(ret)>1:
    random.shuffle(ret)
    

  return ret, (deltaX,deltaY)

def shipyard_actions(board):
  #spawn a ship as long as there is no ship already moved to this shipyard
  for sy in me.shipyards:
    if turn.num_ships < turn.max_ships:
      if (turn.total_halite >= 500) and (sy.position not in turn.taken):
        #spawn one
        sy.next_action = ShipyardAction.SPAWN
        turn.taken[sy.position]=1
        turn.num_ships+=1
        turn.total_halite-=500

def gen_enemy_halite_matrix(board):
  #generate matrix of enemy positions:
  #EP=presence of enemy ship
  #EH=amount of halite in enemy ships
  #ES=presence of enemy shipyards
  EP=np.zeros((size,size))
  EH=np.zeros((size,size))
  ES=np.zeros((size,size))
  for id,ship in board.ships.items():
    if ship.player_id != me.id:
      EH[ship.position.y,ship.position.x]=ship.halite
      EP[ship.position.y,ship.position.x]=1
  for id, sy in board.shipyards.items():
    if sy.player_id != me.id:
      ES[sy.position.y,sy.position.x]=1
  return EP,EH,ES

def dist(a,b):
  #Manhattan distance of the Point difference a to b, considering wrap around
  action,step=dirs_to(a, b, size=21) 
  return abs(step[0]) + abs(step[1])

def nearest_shipyard(pos):
  #return distance, position of nearest shipyard to pos.  100,None if no shipyards
  mn=30
  best_pos=None
  for sy in me.shipyards:
    d=dist(pos, sy.position)
    if d<mn:
      mn=d
      best_pos=sy.position
  return mn,best_pos
  
def assign_targets(board,ships):
  #Assign the ships to a cell containing halite optimally
  #set ship_target[ship_id] to a Position
  #We assume that we mine halite containing cells optimally or return to deposit
  #directly if that is optimal, based on maximizing halite per step.
  #Make a list of pts containing cells we care about, this will be our columns of matrix C
  #the rows are for each ship in collect
  #computes global dict ship_tagert with shipid->Position for its target
  #global ship targets should already exist
  old_target=copy.copy(ship_target)
  ship_target.clear()
  if len(ships)==0:
    return
  halite_min=50
  pts1=[]
  pts2=[]
  for pt,c in board.cells.items():
    assert isinstance(pt,Point)
    if c.halite > halite_min:
      pts1.append(pt)
  #Now add duplicates for each shipyard - this is directly going to deposit
  for sy in me.shipyards:
    for i in range(num_shipyard_targets):
      pts2.append(sy.position)
  #this will be the value of assigning C[ship,pt]
  C=np.zeros((len(ships),len(pts1)+len(pts2)))
  #this will be the optimal mining steps we calculated
  for i,ship in enumerate(ships):
    for j,pt in enumerate(pts1+pts2):
      #two distances: from ship to halite, from halite to nearest shipyard
      d1=dist(ship.position,pt)
      d2,shipyard_position=nearest_shipyard(pt)
      if shipyard_position is None:
        #don't know where to go if no shipyard
        d2=1
      #value of target is based on the amount of halite per turn we can do
      my_halite=ship.halite
      if j < len(pts1):
        #in the mining section
        v, mining=halite_per_turn(my_halite,board.cells[pt].halite, d1+d2)
        #mining is no longer 0, due to min_mine (default)
      else:
        #in the direct to shipyard section
        if d1>0:
          v=my_halite/d1
        else:
          #we are at a shipyard
          v=0
      if board.cells[pt].ship and board.cells[pt].ship.player_id != me.id:
        #if someone else on the cell, see how much halite they have
        #enemy ship
        enemy_halite=board.cells[pt].ship.halite
        if enemy_halite <= my_halite:
          v = -1000   # don't want to go there
        else:
          if d1<3:
            #attack or scare off if reasonably quick to get there
            v+= enemy_halite/(d1+1)  # want to attack them or scare them off
      #print('shipid {} col {} is {} with {:8.1f} score {:8.2f}'.format(ship.id,j, pt,board.cells[pt].halite,v))
      C[i,j]=v
  print('C is {}'.format(C.shape))
  #Compute the optimal assignment
  row,col=scipy.optimize.linear_sum_assignment(C, maximize=True)
  #so ship row[i] is assigned to target col[j]
  #print('got row {} col {}'.format(row,col))
  #print(C[row[0],col[0]])
  pts=pts1+pts2
  for r,c in zip(row,col):
    ship_target[ships[r].id]=pts[c]
  #print out results
  print('\nShip Targets')
  print('Ship      position          target')
  for id,t in ship_target.items():
    st=''
    ta=''
    if board.ships[id].position==t:
      st='MINE'
    elif len(me.shipyards)>0 and t==me.shipyards[0].position:
      st='SHIPYARD'
    if id not in old_target or old_target[id] != ship_target[id]:
      ta=' NEWTARGET'
    print('{0:6}  at ({1[0]:2},{1[1]:2})  assigned ({2[0]:2},{2[1]:2}) h {3:3} {4:10} {5:10}'.format(
      id, board.ships[id].position, t, board.cells[t].halite,st, ta))

  return

def make_avoidance_matrix(myship_halite):
  #make a matrix of True where we want to avoid, uses
  #turn.EP=enemy position matrix
  #turn.EH=enemy halite matrix
  #turn.ES=enemy shipyard matrix
  filter=np.array([[0,1,0],[1,1,1],[0,1,0]])
  bad_ship=np.logical_and(turn.EH <= myship_halite,turn.EP)
  avoid=scipy.ndimage.convolve(bad_ship, filter, mode='wrap',cval=0.0)
  #avoid=np.logical_or(avoid,turn.ES)
  return avoid

def make_attack_matrix(myship_halite):
  #make a matrix of True where we would want to move to attack an enemy ship
  #for now, we just move to where the ship is.
  #turn.EP=enemy position matrix
  #turn.EH=enemy halite matrix
  #turn.ES=enemy shipyard matrix
  nextEH = ((np.roll(turn.EH,-1,1)==np.roll(prev_turn.EH,-2,1))*np.roll(turn.EH,-1,1)+  # moving west
            (np.roll(turn.EH,1,1)==np.roll(prev_turn.EH,2,1))*np.roll(turn.EH,1,1)+     # moving east
            (np.roll(turn.EH,-1,0)==np.roll(prev_turn.EH,-2,0))*np.roll(turn.EH,-1,0)+  # moving south
            (np.roll(turn.EH,1,0)==np.roll(prev_turn.EH,2,0))*np.roll(turn.EH,1,0)+     # moving north
            (turn.ES*50))                                                               # enemy shipyard

  attack=(nextEH > myship_halite)
  #print('attack',attack)
  return attack

def get_max_halite_ship(board, avoid_danger=True):
  #Return my Ship carrying max halite, or None if no ships
  #NOTE: creating avoid matrix again!
  mx=-1
  the_ship=None
  for ship in me.ships:
    x=ship.position.x
    y=ship.position.y
    avoid=make_avoidance_matrix(ship.halite)
    if ship.halite>mx and (not avoid_danger or not avoid[y,x]):
      mx=ship.halite
      the_ship=ship
  return the_ship

def remove_dups(p):
  #remove duplicates from a list without changing order
  #Not efficient for long lists
  ret=[]
  for x in p:
    if x not in ret:
      ret.append(x)
  return ret

def matrix_lookup(matrix,pos):
  return matrix[pos.y,pos.x]

def ship_converts(board):
  #if no shipyard, convert the ship carrying max halite unless it is in danger
  if turn.num_shipyards==0 and not turn.last_episode:
    mx=get_max_halite_ship(board)
    if mx is not None:
      if mx.halite + turn.total_halite > 500:
        mx.next_action=ShipAction.CONVERT
        turn.taken[mx.position]=1
        turn.num_shipyards+=1
        turn.total_halite-=500
  #Now check the rest to see if they should convert
  for ship in me.ships:
    if ship.next_action:
      continue
    #CHECK if in danger without escape, convert if h>500
    avoid=make_avoidance_matrix(ship.halite)
    z=[matrix_lookup(avoid,move(ship.position,a)) for a in all_actions]
    if np.all(z) and ship.halite > 500:
      ship.next_action=ShipAction.CONVERT
      turn.taken[ship.position]=1
      turn.num_shipyards+=1
      turn.total_halite-=500
      print('ship id {} no escape converting'.format(ship.id))
    #CHECK if last step and > 500 halite, convert
    if turn.last_episode and ship.halite > 500:
      ship.next_action=ShipAction.CONVERT
      turn.taken[ship.position]=1
      turn.num_shipyards+=1
      turn.total_halite-=500
    #CHECK if we're hauling long distance without threats
    nearSY,tmp = nearest_shipyard(ship.position)
    if nearSY>9 and turn.num_shipyards<2 and ship.halite+turn.total_halite > 750 and not np.any(z):
      ship.next_action=ShipAction.CONVERT
      turn.taken[ship.position]=1
      turn.num_shipyards+=1
      turn.total_halite-=500
                               
      
def ship_moves(board):
  ships=[ship for ship in me.ships if ship.next_action is None]
  #update ship_target
  assign_targets(board,ships)
  #For all ships without a target, we give them a random movement (we will check below if this
  actions={}   # record actions for each ship
  for ship in ships:
    if ship.id in ship_target:
      a,delta = dirs_to(ship.position, ship_target[ship.id],size=size)
      actions[ship.id]=a
    else:
      actions[ship.id]=[random.choice(all_actions)]
      
  for ship in ships:
    action=None
    x=ship.position
    #generate matrix of places to attack and places to avoid
    avoid=make_avoidance_matrix(ship.halite)
    attack=make_attack_matrix(ship.halite)
    #see if there is a attack options
    action_list=actions[ship.id]+[None]+all_actions
    #see if we should add an attack diversion to our options
    #NOTE: we will avoid attacking a ship that is on the avoid spot - is this a good idea?
    for a in all_actions:
      m=move(x,a)
      if attack[m.y,m.x]:
        print('ship id {} attacking {}'.format(ship.id,a))
        action_list.insert(0,a)
        break
    #now try the options, but don't bother repeating any
    action_list=remove_dups(action_list)
    for a in action_list:
      m=move(x,a)
      if avoid[m.y,m.x]:
        print('ship id {} avoiding {}'.format(ship.id,a))
      if m not in turn.taken and not avoid[m.y,m.x]:
        action=a
        break
    ship.next_action=action
    turn.taken[m]=1
    
# Returns the commands we send to our ships and shipyards, must be last function in file
def agent(obs, config):
  global size
  global start
  global prev_board
  global me
  global did_init
  #Do initialization 1 time
  start_step=time.time()
  if start is None:
    start=time.time()
  if not did_init:
    init(obs,config)
    prev_board = Board(obs, config)
    did_init=True
  board = Board(obs, config)
  me=board.current_player
  set_turn_data(board)
  set_prev_turn_data(prev_board)
  print('==== step {} sim {}'.format(board.step,board.step+1))
  print('ships {} shipyards {}'.format(turn.num_ships,turn.num_shipyards))
  print_enemy_ships(board)
  ship_converts(board)
  ship_moves(board)
  shipyard_actions(board)
  print_actions(board)
  prev_board = board
  print('time this turn: {:8.3f} total elapsed {:8.3f}'.format(time.time()-start_step,time.time()-start))
  return me.next_actions

In [ ]:
%%writefile htt3.py
import time
import copy
import sys
import math
import collections
import pprint
import numpy as np
import scipy.optimize
import scipy.ndimage
from kaggle_environments.envs.halite.helpers import *
import kaggle_environments
import random
'''


'''
CONFIG_MAX_SHIPS=28

#print('kaggle version',kaggle_environments.__version__)
#### Global variables
all_actions=[ShipAction.NORTH, ShipAction.EAST,ShipAction.SOUTH,ShipAction.WEST]
all_dirs=[Point(0,1), Point(1,0), Point(0,-1), Point(-1,0)]
start=None
num_shipyard_targets=4
size=None
# Will keep track of whether a ship is collecting halite or carrying cargo to a shipyard
ship_target={}   # where to go to collect
me=None
did_init=False
quiet=False
C=None
class Obj:
  pass
# will hold global data for this turn, updating as we set actions.
# E.g. number of ships, amount of halite
# taking into account the actions set so far.  Also max_ships, etc.
turn=Obj()

prev_turn=Obj()


### Data
#turns_optimal[CHratio, round_trip_travel] for mining
#See notebook on optimal mining kaggle.com/solverworld/optimal-mining
turns_optimal=np.array(
  [[0, 2, 3, 4, 4, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8],
   [0, 1, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7],
   [0, 0, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7],
   [0, 0, 1, 2, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6],
   [0, 0, 0, 1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6],
   [0, 0, 0, 0, 0, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5],
   [0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

#### Functions
def print_enemy_ships(board):
  print('\nEnemy Ships')
  for ship in board.ships.values():
    if ship.player_id != me.id:
      print('{:6}  {} halite {}'.format(ship.id,ship.position,ship.halite))
      
def print_actions(board):
  print('\nShip Actions')
  for ship in me.ships:
    print('{:6}  {}  {} halite {}'.format(ship.id,ship.position,ship.next_action,ship.halite))
  print('Shipyard Actions')
  for sy in me.shipyards:
    print('{:6}  {}  {}'.format(sy.id,sy.position,sy.next_action))

def print_none(*args):
  pass

def compute_max_ships(step):
  #This needs to be tuned, perhaps based on amount of halite left
  if step < 200:
    return CONFIG_MAX_SHIPS
  elif step < 300:
    return CONFIG_MAX_SHIPS-6
  elif step < 350:
    return CONFIG_MAX_SHIPS-8
  else:
    return CONFIG_MAX_SHIPS-10

def set_turn_data(board):
  #initialize the global turn data for this turn
  turn.num_ships=len(me.ships)
  turn.max_ships=compute_max_ships(board.step)
  turn.total_halite=me.halite
  #this is matrix of halite in cells
  turn.halite_matrix=np.reshape(board.observation['halite'], (board.configuration.size,board.configuration.size))
  turn.num_shipyards=len(me.shipyards)
  #compute enemy presence and enemy halite matrices
  turn.EP,turn.EH,turn.ES=gen_enemy_halite_matrix(board)
  #filled in by shipid as a ship takes up a square
  turn.taken={}
  turn.last_episode = (board.step == (board.configuration.episode_steps-2))
    
def set_prev_turn_data(prev_board):
  #initialize the global turn data for this turn
  prev_turn.num_ships=len(prev_board.current_player.ships)
  prev_turn.max_ships=compute_max_ships(prev_board.step)
  prev_turn.total_halite=prev_board.current_player.halite
  #this is matrix of halite in cells
  prev_turn.halite_matrix=np.reshape(prev_board.observation['halite'], (prev_board.configuration.size,prev_board.configuration.size))
  prev_turn.num_shipyards=len(prev_board.current_player.shipyards)
  #compute enemy presence and enemy halite matrices
  prev_turn.EP,prev_turn.EH,prev_turn.ES=gen_enemy_halite_matrix(prev_board)
  #filled in by shipid as a ship takes up a square
  prev_turn.taken={}
  prev_turn.last_episode = (prev_board.step == (prev_board.configuration.episode_steps-2))
  
def init(obs,config):
  #This is only called on first call to agent()
  #Do initalization things
  global size
  global print
  if hasattr(config,'myval') and config.myval==9 and not quiet:
    #we are called locally, so leave prints OK
    pass
  else:
    #we are called in competition, quiet output
    print=print_none
    pprint.pprint=print_none
  size = config.size

def limit(x,a,b):
  if x<a:
    return a
  if x>b:
    return b
  return x
  
def num_turns_to_mine(C,H,rt_travel):
  #How many turns should we plan on mining?
  #C=carried halite, H=halite in square, rt_travel=steps to square and back to shipyard
  if C==0:
    ch=0
  elif H==0:
    ch=turns_optimal.shape[0]
  else:
    ch=int(math.log(C/H)*2.5+5.5)
    ch=limit(ch,0,turns_optimal.shape[0]-1)
  rt_travel=int(limit(rt_travel,0,turns_optimal.shape[1]-1))
  return turns_optimal[ch,rt_travel]

def halite_per_turn(carrying, halite,travel,min_mine=1):
  #compute return from going to a cell containing halite, using optimal number of mining steps
  #returns halite mined per turn, optimal number of mining steps
  #Turns could be zero, meaning it should just return to a shipyard (subject to min_mine)
  turns=num_turns_to_mine(carrying,halite,travel)
  if turns<min_mine:
    turns=min_mine
  mined=carrying+(1-.75**turns)*halite
  return mined/(travel+turns), turns
  
def move(pos, action):
  ret=None
  #return new Position from pos when action is applied
  if action==ShipAction.NORTH:
    ret=pos+Point(0,1)
  if action==ShipAction.SOUTH:
    ret=pos+Point(0,-1)
  if action==ShipAction.EAST:
    ret=pos+Point(1,0)
  if action==ShipAction.WEST:
    ret=pos+Point(-1,0)
  if ret is None:
    ret=pos
  #print('move pos {} {} => {}'.format(pos,action,ret))
  return ret % size

def dirs_to(p1, p2, size=21):
  #Get the actions you should take to go from Point p1 to Point p2
  #using shortest direction by wraparound
  #Args: p1: from Point
  #      p2: to Point
  #      size:  size of board
  #returns: list of directions, tuple (deltaX,deltaY)
  #The list is of length 1 or 2 giving possible directions to go, e.g.
  #to go North-East, it would return [ShipAction.NORTH, ShipAction.EAST], because
  #you could use either of those first to go North-East.
  #[None] is returned if p1==p2 and there is no need to move at all
  deltaX, deltaY=p2 - p1
  if abs(deltaX)>size/2:
    #we wrap around
    if deltaX<0:
      deltaX+=size
    elif deltaX>0:
      deltaX-=size
  if abs(deltaY)>size/2:
    #we wrap around
    if deltaY<0:
      deltaY+=size
    elif deltaY>0:
      deltaY-=size
  #the delta is (deltaX,deltaY)
  ret=[]
  if deltaX>0:
    ret.append(ShipAction.EAST)
  if deltaX<0:
    ret.append(ShipAction.WEST)
  if deltaY>0:
    ret.append(ShipAction.NORTH)
  if deltaY<0:
    ret.append(ShipAction.SOUTH)
  if len(ret)==0:
    ret=[None]  # do not need to move at all
    
  if len(ret)>1:
    random.shuffle(ret)
    

  return ret, (deltaX,deltaY)

def shipyard_actions(board):
  #spawn a ship as long as there is no ship already moved to this shipyard
  for sy in me.shipyards:
    if turn.num_ships < turn.max_ships:
      if (turn.total_halite >= 500) and (sy.position not in turn.taken):
        #spawn one
        sy.next_action = ShipyardAction.SPAWN
        turn.taken[sy.position]=1
        turn.num_ships+=1
        turn.total_halite-=500

def gen_enemy_halite_matrix(board):
  #generate matrix of enemy positions:
  #EP=presence of enemy ship
  #EH=amount of halite in enemy ships
  #ES=presence of enemy shipyards
  EP=np.zeros((size,size))
  EH=np.zeros((size,size))
  ES=np.zeros((size,size))
  for id,ship in board.ships.items():
    if ship.player_id != me.id:
      EH[ship.position.y,ship.position.x]=ship.halite
      EP[ship.position.y,ship.position.x]=1
  for id, sy in board.shipyards.items():
    if sy.player_id != me.id:
      ES[sy.position.y,sy.position.x]=1
  return EP,EH,ES

def dist(a,b):
  #Manhattan distance of the Point difference a to b, considering wrap around
  action,step=dirs_to(a, b, size=21) 
  return abs(step[0]) + abs(step[1])

def nearest_shipyard(pos):
  #return distance, position of nearest shipyard to pos.  100,None if no shipyards
  mn=30
  best_pos=None
  for sy in me.shipyards:
    d=dist(pos, sy.position)
    if d<mn:
      mn=d
      best_pos=sy.position
  return mn,best_pos
  
def assign_targets(board,ships):
  #Assign the ships to a cell containing halite optimally
  #set ship_target[ship_id] to a Position
  #We assume that we mine halite containing cells optimally or return to deposit
  #directly if that is optimal, based on maximizing halite per step.
  #Make a list of pts containing cells we care about, this will be our columns of matrix C
  #the rows are for each ship in collect
  #computes global dict ship_tagert with shipid->Position for its target
  #global ship targets should already exist
  old_target=copy.copy(ship_target)
  ship_target.clear()
  if len(ships)==0:
    return
  halite_min=50
  pts1=[]
  pts2=[]
  for pt,c in board.cells.items():
    assert isinstance(pt,Point)
    if c.halite > halite_min:
      pts1.append(pt)
  #Now add duplicates for each shipyard - this is directly going to deposit
  for sy in me.shipyards:
    for i in range(num_shipyard_targets):
      pts2.append(sy.position)
  #this will be the value of assigning C[ship,pt]
  C=np.zeros((len(ships),len(pts1)+len(pts2)))
  #this will be the optimal mining steps we calculated
  for i,ship in enumerate(ships):
    for j,pt in enumerate(pts1+pts2):
      #two distances: from ship to halite, from halite to nearest shipyard
      d1=dist(ship.position,pt)
      d2,shipyard_position=nearest_shipyard(pt)
      if shipyard_position is None:
        #don't know where to go if no shipyard
        d2=1
      #value of target is based on the amount of halite per turn we can do
      my_halite=ship.halite
      if j < len(pts1):
        #in the mining section
        v, mining=halite_per_turn(my_halite,board.cells[pt].halite, d1+d2)
        #mining is no longer 0, due to min_mine (default)
      else:
        #in the direct to shipyard section
        if d1>0:
          v=my_halite/d1
        else:
          #we are at a shipyard
          v=0
      if board.cells[pt].ship and board.cells[pt].ship.player_id != me.id:
        #if someone else on the cell, see how much halite they have
        #enemy ship
        enemy_halite=board.cells[pt].ship.halite
        if enemy_halite <= my_halite:
          v = -1000   # don't want to go there
        else:
          if d1<3:
            #attack or scare off if reasonably quick to get there
            v+= enemy_halite/(d1+1)  # want to attack them or scare them off
      #print('shipid {} col {} is {} with {:8.1f} score {:8.2f}'.format(ship.id,j, pt,board.cells[pt].halite,v))
      C[i,j]=v
  print('C is {}'.format(C.shape))
  #Compute the optimal assignment
  row,col=scipy.optimize.linear_sum_assignment(C, maximize=True)
  #so ship row[i] is assigned to target col[j]
  #print('got row {} col {}'.format(row,col))
  #print(C[row[0],col[0]])
  pts=pts1+pts2
  for r,c in zip(row,col):
    ship_target[ships[r].id]=pts[c]
  #print out results
  print('\nShip Targets')
  print('Ship      position          target')
  for id,t in ship_target.items():
    st=''
    ta=''
    if board.ships[id].position==t:
      st='MINE'
    elif len(me.shipyards)>0 and t==me.shipyards[0].position:
      st='SHIPYARD'
    if id not in old_target or old_target[id] != ship_target[id]:
      ta=' NEWTARGET'
    print('{0:6}  at ({1[0]:2},{1[1]:2})  assigned ({2[0]:2},{2[1]:2}) h {3:3} {4:10} {5:10}'.format(
      id, board.ships[id].position, t, board.cells[t].halite,st, ta))

  return

def make_avoidance_matrix(myship_halite):
  #make a matrix of True where we want to avoid, uses
  #turn.EP=enemy position matrix
  #turn.EH=enemy halite matrix
  #turn.ES=enemy shipyard matrix
  filter=np.array([[0,1,0],[1,1,1],[0,1,0]])
  bad_ship=np.logical_and(turn.EH <= myship_halite,turn.EP)
  avoid=scipy.ndimage.convolve(bad_ship, filter, mode='wrap',cval=0.0)
  #avoid=np.logical_or(avoid,turn.ES)
  return avoid

def make_attack_matrix(myship_halite):
  #make a matrix of True where we would want to move to attack an enemy ship
  #for now, we just move to where the ship is.
  #turn.EP=enemy position matrix
  #turn.EH=enemy halite matrix
  #turn.ES=enemy shipyard matrix
  nextEH = ((np.roll(turn.EH,-1,1)==np.roll(prev_turn.EH,-2,1))*np.roll(turn.EH,-1,1)+  # moving west
            (np.roll(turn.EH,1,1)==np.roll(prev_turn.EH,2,1))*np.roll(turn.EH,1,1)+     # moving east
            (np.roll(turn.EH,-1,0)==np.roll(prev_turn.EH,-2,0))*np.roll(turn.EH,-1,0)+  # moving south
            (np.roll(turn.EH,1,0)==np.roll(prev_turn.EH,2,0))*np.roll(turn.EH,1,0)+     # moving north
            (turn.ES*50))                                                               # enemy shipyard

  attack=(nextEH > myship_halite)
  #print('attack',attack)
  return attack

def get_max_halite_ship(board, avoid_danger=True):
  #Return my Ship carrying max halite, or None if no ships
  #NOTE: creating avoid matrix again!
  mx=-1
  the_ship=None
  for ship in me.ships:
    x=ship.position.x
    y=ship.position.y
    avoid=make_avoidance_matrix(ship.halite)
    if ship.halite>mx and (not avoid_danger or not avoid[y,x]):
      mx=ship.halite
      the_ship=ship
  return the_ship

def remove_dups(p):
  #remove duplicates from a list without changing order
  #Not efficient for long lists
  ret=[]
  for x in p:
    if x not in ret:
      ret.append(x)
  return ret

def matrix_lookup(matrix,pos):
  return matrix[pos.y,pos.x]

def ship_converts(board):
  #if no shipyard, convert the ship carrying max halite unless it is in danger
  if turn.num_shipyards==0 and not turn.last_episode:
    mx=get_max_halite_ship(board)
    if mx is not None:
      if mx.halite + turn.total_halite > 500:
        mx.next_action=ShipAction.CONVERT
        turn.taken[mx.position]=1
        turn.num_shipyards+=1
        turn.total_halite-=500
  #Now check the rest to see if they should convert
  for ship in me.ships:
    if ship.next_action:
      continue
    #CHECK if in danger without escape, convert if h>500
    avoid=make_avoidance_matrix(ship.halite)
    z=[matrix_lookup(avoid,move(ship.position,a)) for a in all_actions]
    if np.all(z) and ship.halite > 500:
      ship.next_action=ShipAction.CONVERT
      turn.taken[ship.position]=1
      turn.num_shipyards+=1
      turn.total_halite-=500
      print('ship id {} no escape converting'.format(ship.id))
    #CHECK if last step and > 500 halite, convert
    if turn.last_episode and ship.halite > 500:
      ship.next_action=ShipAction.CONVERT
      turn.taken[ship.position]=1
      turn.num_shipyards+=1
      turn.total_halite-=500
    #CHECK if we're hauling long distance without threats
    nearSY,tmp = nearest_shipyard(ship.position)
    if nearSY>9 and turn.num_shipyards<2 and ship.halite+turn.total_halite > 750 and not np.any(z):
      ship.next_action=ShipAction.CONVERT
      turn.taken[ship.position]=1
      turn.num_shipyards+=1
      turn.total_halite-=500
                               
      
def ship_moves(board):
  ships=[ship for ship in me.ships if ship.next_action is None]
  #update ship_target
  assign_targets(board,ships)
  #For all ships without a target, we give them a random movement (we will check below if this
  actions={}   # record actions for each ship
  for ship in ships:
    if ship.id in ship_target:
      a,delta = dirs_to(ship.position, ship_target[ship.id],size=size)
      actions[ship.id]=a
    else:
      actions[ship.id]=[random.choice(all_actions)]
      
  for ship in ships:
    action=None
    x=ship.position
    #generate matrix of places to attack and places to avoid
    avoid=make_avoidance_matrix(ship.halite)
    attack=make_attack_matrix(ship.halite)
    #see if there is a attack options
    action_list=actions[ship.id]+[None]+all_actions
    #see if we should add an attack diversion to our options
    #NOTE: we will avoid attacking a ship that is on the avoid spot - is this a good idea?
    for a in all_actions:
      m=move(x,a)
      if attack[m.y,m.x]:
        print('ship id {} attacking {}'.format(ship.id,a))
        action_list.insert(0,a)
        break
    #now try the options, but don't bother repeating any
    action_list=remove_dups(action_list)
    for a in action_list:
      m=move(x,a)
      if avoid[m.y,m.x]:
        print('ship id {} avoiding {}'.format(ship.id,a))
      if m not in turn.taken and not avoid[m.y,m.x]:
        action=a
        break
    ship.next_action=action
    turn.taken[m]=1
    
# Returns the commands we send to our ships and shipyards, must be last function in file
def agent(obs, config):
  global size
  global start
  global prev_board
  global me
  global did_init
  #Do initialization 1 time
  start_step=time.time()
  if start is None:
    start=time.time()
  if not did_init:
    init(obs,config)
    prev_board = Board(obs, config)
    did_init=True
  board = Board(obs, config)
  me=board.current_player
  set_turn_data(board)
  set_prev_turn_data(prev_board)
  print('==== step {} sim {}'.format(board.step,board.step+1))
  print('ships {} shipyards {}'.format(turn.num_ships,turn.num_shipyards))
  print_enemy_ships(board)
  ship_converts(board)
  ship_moves(board)
  shipyard_actions(board)
  print_actions(board)
  prev_board = board
  print('time this turn: {:8.3f} total elapsed {:8.3f}'.format(time.time()-start_step,time.time()-start))
  return me.next_actions

In [ ]:
#env.run(["submission.py", "submission.py", "submission.py", "submission.py"])
env.run(["submission.py", "swarm.py", "optimus.py", "random"])
env.render(mode="ipython", width=800, height=600)